# Fase 1 del proyecto

En esta primera fase se limpiará los datos y se elegirá una problemática para brindar solución.

In [ ]:
'''
Download necessary libraries if not have them already
'''
# !pip install numpy
# !pip install matplotlib.pyplot
# !pip install seaborn
# !pip install scikit-learn 
# !pip install pandas

## Exploración de datos
Primero se va a chequear si existe datos nulos en los csv y recabar la cantidad de datos nullos encontrados por cada .csv del conjunto de datos. De igual forma
se verá qué tipos de datos se maneja y ver cantidatos para aplicar transformación a datos enteros. 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Callable, List, Tuple
import os
import glob

In [ ]:
# these functions will help with the data analysis 
# there's chance to add more information if deemeded necessary
def get_null_count(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Gives a count of nulls per column and its relative percentage.
    It does not change or replace nulls.
    params: 
        df: dataframe to get null info of
    returns:
        summary: dataframe that has a null count per col and a relative % of nulls in that col
    '''
    nulls: pd.Series = df.isnull().sum() # get the count of nulls per col
    percent: pd.Series = (nulls / len(df)) * 100 # get the percentage of nulls 
    summary: pd.DataFrame = pd.DataFrame({
        "Nulls" : nulls,
        "Percentage" : percent.round(2)
    })
    return summary[summary['Nulls'] > 0].sort_values(by='Nulls', ascending=False) # sort values by Nulls col , can change ascending to true

def get_types(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Gives the datatypes found within a dataframe.
    It does not change datatypes.
    params: 
        df: dataframe to get types of
    returns:
        types: dataframe with the type of each column
    '''
    types: pd.DataFrame = pd.DataFrame({
        'col_name' : df.columns,
        'data_type' : df.dtypes.astype(str).values
    })
    return types

def get_row_and_col_count(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Gives the number of rows and columns found in a dataframe
    params:
        df: dataframe to get num of cols and rows
    returns:
        num_row_col: dataframe with the number of columns and rows
    '''
    num_row_col: pd.DataFrame = pd.DataFrame([{
        'num_cols': len(df.columns),
        'num_rows': len(df)
    }])
    return num_row_col

def get_descriptive_stats(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Gives a brief description of the dataframe
    '''
    return df.describe().transpose()

def get_unique(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Return unique values found in the dataframe
    params:
        df: dataframe to get unique values from
    return:
        dataframe with unique values
    '''
    return df.nunique().to_frame(name='Unique Values')

def get_top_frequent(df: pd.DataFrame, top_n: int = 3) -> pd.DataFrame:
    '''
    Returns top 10 most frequent values found in the dataframe
    params: 
        df: dataframe to get unique values
        top_n: top number of values to get
    '''
    rows:list = []
    for col in df.select_dtypes(include='object').columns:
        top_vals = df[col].value_counts().head(top_n) # get the top 3 values
        for val, freq in top_vals.items(): # from those values, get the top value and its frequency
            rows.append({
                'Column Name': col,
                'Top Value': str(val), 
                'Frequency': freq
            })
    return pd.DataFrame(rows)

def write_report(df: pd.DataFrame, df_name: str, path: str, summary_functions: List[Callable]) -> None:
    '''
    Method that writes a report into a .txt of a certain dataframe
    params: 
        df: dataframe to be read
        path: where the .txt log will be saved
        summary_functions: a list functions that will be run and its returns be saved into the report
    returns:
        None
    '''
    if summary_functions is None:
        print("Please provide functions to be able to write a summary")
    
    with open(path, 'w') as log:
        log.write(f"DataFrame Summary Report Of {df_name}\n") # write header
        log.write("=" * 80 + "\n\n") 

        # for each function in the summary_functions list, write its summary and results
        for func in summary_functions:
            section_title: str = func.__name__.replace('_', ' ').title() # get the function name as title
            log.write(f"{section_title}\n")
            log.write("-" * len(section_title) + "\n")
            result_df: pd.DataFrame = func(df) # pass the dataframe argument to the function
            if result_df.empty:
                log.write("No information to report\n\n")
            else:
                log.write(result_df.to_string(index=True))
                log.write("\n\n")
        
        log.write("=" * 80 + "\nEnd\n") # finish the report


def generate_report_folder(csv_folder: str, report_folder: str, summary_functions: List[Callable]) -> None:
    '''
    Generates a report folder based upon the functions
    params:
        csv_folder: where the csv files are 
        report_folder: final folder where reports will be stored
        summary_functions: functions' returns that will be stored in the report
    '''
    os.makedirs(report_folder, exist_ok=True) # create folder if it does not exist
    csv_files: list = glob.glob(os.path.join(csv_folder, '*.csv'))
    for csv_path in csv_files:
        try:
            df: pd.DataFrame = pd.read_csv(csv_path)
            file_name: str = os.path.splitext(os.path.basename(csv_path))[0]
            report_path: str = os.path.join(report_folder, f"{file_name}_raw_report.txt")
            write_report(df=df, df_name=file_name, path=report_path, summary_functions=summary_functions)
            print(f"Report written into: {report_path}")
        except Exception as e:
            print(f"Failed processing {csv_path}: {e}")



In [10]:
generate_report_folder(csv_folder="../dataset", report_folder="../dataset_reports_raw", summary_functions=[get_null_count, get_types, get_row_and_col_count, get_descriptive_stats, 
                                                                                                           get_top_frequent, get_unique])

Report written into: ../dataset_reports_raw\olist_customers_dataset_raw_report.txt
Report written into: ../dataset_reports_raw\olist_geolocation_dataset_raw_report.txt
Report written into: ../dataset_reports_raw\olist_orders_dataset_raw_report.txt
Report written into: ../dataset_reports_raw\olist_order_items_dataset_raw_report.txt
Report written into: ../dataset_reports_raw\olist_order_payments_dataset_raw_report.txt
Report written into: ../dataset_reports_raw\olist_order_reviews_dataset_raw_report.txt
Report written into: ../dataset_reports_raw\olist_products_dataset_raw_report.txt
Report written into: ../dataset_reports_raw\olist_sellers_dataset_raw_report.txt
Report written into: ../dataset_reports_raw\product_category_name_translation_raw_report.txt


## Análisis estadístico

## Transformación de datos
Al observar que hay ciertos conjuntos de datos con varios datos faltantes como en olist_order_reviews. Donde casi un 80% de las reseñas están vacías. Hay otras que también están vacías pero son cercanas a menos del 5%. Por lo que podemos decidir prescindir de estas o no. Ahora bien, debemos ver si realmente utilizaremos los reviews del dataset para nuestro fin o si no. 

In [ ]:
from sklearn.preprocessing import OneHotEncoder